# Wideband Statecount and Histogram Data Collection

## 1 Setup

In [ ]:
import os
import csv
import pathlib
import shutil

import numpy as np
from tabulate import tabulate
from PyTango import DeviceProxy

import notebook_tools.histogram_client as HistogramClient
from notebook_tools.misc_helper import get_tango_host

First, get the ns to check and set to be used at the tango host for future steps.

In [ ]:
!kubectl get ns | grep ska-mid-psi

Set the boards, lanes, and namespace values as required, with "namespace" pointing to a namespace that has completed a scan:

In [ ]:
namespace = "ci-ska-mid-psi-1720589809-alexschell"  # The namespace to connect to
boards = [9, 10]  # The talon boards to use when checking for data
lanes = [0, 1]  # The lanes to check (0-3)

# These values should not need to be changed
timeout_ms = 3000  # Max time allowed for trying to access histograms
path = os.getcwd() + "/output-data/" + namespace

To grab the histogram client and access devices, we set the TANGO host environment variable to point to the namespace:

In [ ]:
os.environ["TANGO_HOST"] = get_tango_host(namespace)

Set up the holders for the histogram and statecount data:

In [ ]:
pre_vcc_collected = False
pre_vcc_x = []
pre_vcc_y = []

post_vcc_collected = False
post_vcc_x_lanes = [[] for x in range(4)]
post_vcc_y_lanes = [[] for x in range(4)]

post_16k_collected = False
post_16k_x_lanes = [[] for x in range(4)]
post_16k_y_lanes = [[] for x in range(4)]

state_count_vec_1 = []
state_count_vec_2 = []
power_spectrum_1 = []
power_spectrum_2 = []

In [ ]:
# Mapping higher number boards to correct value
if len(boards) > 1 and boards[1] == 17:
    boards[1] = 2
    print("Replacing talon17 with talon2 in the target_boards")

if boards[0] >= 5:
    print("Mapping talons of higher numbers to 1-4")
    boards = list(map(lambda x: x - (((x - 1) // 4) * 4), boards))

### 1.1 Setup Directories

To store the files we will generate and for compression, generate a holder directory and subdirectories for the chosen boards:

In [ ]:
# part folder for namesapce
folder = pathlib.Path(path)
folder.mkdir(parents=True, exist_ok=True)
# Subfolder for each board
for board in boards:
    board_folder = path + f"/talondx-00{board}/"
    subfolder = pathlib.Path(board_folder)
    subfolder.mkdir(parents=True, exist_ok=True)
    histogram_folder = board_folder + "histograms"
    wb_folder = board_folder + "statecount"

## 2 Histogram Data Collection

The following stages collect the histogram data from the respective stages in the signal chain, each being able to be run independently.

### 2.1 Pre-VCC

To collect the pre-VCC data, step through each of the boards defined, and collect the histogram from each:

In [ ]:
for board in boards:
    print(f"----Board #{board}----")
    device = f"talondx-00{board}/histogram/e_pre_vcc"
    # Create a histogram client to the VCC device
    client = HistogramClient.HistogramClient(device, timeout_ms)
    # Run the capture command on the device and read the data for the histogram
    success, data = client.capture()
    if success:
        pre_vcc_collected = True
        # If we captured the data, store the x and y histograms as 2D lists
        print("Pre-VCC data capture successful, storing")
        pre_vcc_x = data[0]
        pre_vcc_y = data[1]
        print(f"Pol-x Histogram is of dimensions {len(pre_vcc_x)}X{len(pre_vcc_x[0])}")
        print(f"Pol-y Histogram is of dimensions {len(pre_vcc_y)}X{len(pre_vcc_y[0])}")
    else:
        print("Could not get Pre-VCC data!")

To display the histogram data if desired:

In [ ]:
print("Pol X histogram data:")
pol_x_headers = range(len(pre_vcc_x))
print(tabulate(pre_vcc_x, tablefmt="simple", headers=pol_x_headers, showindex="always"))
print("Pol Y histogram data:")
pol_y_headers = range(len(pre_vcc_y))
print(tabulate(pre_vcc_y, tablefmt="simple", headers=pol_y_headers, showindex="always"))

### 2.2 Post-VCC

To collect the post-vcc histogram data, we do a similar check as we did with the pre-vcc. However, as there are now multiple lanes of data, we also iterate through them to collect each that has been defined:

In [ ]:
for board in boards:
    print(f"----Board #{board}----")
    for lane in lanes:
        device = f"talondx-00{board}/histogram/e_post_vcc_{lane}"
        client = HistogramClient.HistogramClient(device, timeout_ms)
        success, data = client.capture()
        if success:
            post_vcc_collected = True
            print(f"Captured data for Board {board}, lane {lane}")
            post_vcc_x_lanes[lane] = data[0]
            post_vcc_y_lanes[lane] = data[1]
            print(f"Pol-x Histogram is of dimensions {len(data[0])}X{len(data[0][0])}")
            print(f"Pol-y Histogram is of dimensions {len(data[1])}X{len(data[1][0])}")
        else:
            print(f"Could not capture data for Board {board}, lane {lane}")

### 2.3 16K Channelizer

To collect the 16k Channelizer histogram data, we run through as above:

In [ ]:
for board in boards:
    print(f"----Board #{board}----")
    for lane in lanes:
        device = f"talondx-00{board}/histogram/e_post_ch16k_{lane}"
        client = HistogramClient.HistogramClient(device, timeout_ms)
        success, data = client.capture()
        if success:
            post_16k_collected = True
            print(f"Captured data for Board {board}, lane {lane}")
            post_16k_x_lanes[lane] = data[0]
            post_16k_y_lanes[lane] = data[1]
            print(f"Pol-x Histogram is of dimensions {len(data[0])}X{len(data[0][0])}")
            print(f"Pol-y Histogram is of dimensions {len(data[1])}X{len(data[1][0])}")
        else:
            print(f"Could not capture data for Board {board}, lane {lane}")

### 2.4 Store Histogram Data

Once the desired histograms are collected, write them to the relevant folders and csv files:

In [ ]:
for board in boards:
    print(f"----Board #{board}----")
    histogram_folder = path + f"/talondx-00{board}/" + "/histograms/"
    # Generate the folder if not present
    folder = pathlib.Path(histogram_folder)
    folder.mkdir(parents=True, exist_ok=True)
    pre_vcc_folder = pathlib.Path(histogram_folder + "/pre_vcc/")
    post_vcc_folder = pathlib.Path(histogram_folder + "/post_vcc/")
    post_16k_channelizer_folder = pathlib.Path(histogram_folder + "/post_16k/")

    # Write each CSV file under this folder if the data was collected
    # Pre-vcc
    if pre_vcc_collected:
        pre_vcc_folder.mkdir(parents=True, exist_ok=True)
        with open(str(pre_vcc_folder) + "/pre_vcc_x.csv", "w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(pre_vcc_x)
            print(f"Wrote histogram data to {histogram_folder}pre_vcc_x.csv")
        with open(str(pre_vcc_folder) + "/pre_vcc_y.csv", "w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(pre_vcc_y)
            print(f"Wrote histogram data to {histogram_folder}pre_vcc_y.csv")

    # Post-vcc
    if post_vcc_collected:
        post_vcc_folder.mkdir(parents=True, exist_ok=True)
        for lane in lanes:
            with open(str(post_vcc_folder) + f"/post_vcc_x_lane{lane}.csv", "w") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(post_vcc_x_lanes[lane])
                print(f"Wrote histogram data to {post_vcc_folder}post_vcc_x.csv")
            with open(str(post_vcc_folder) + f"/post_vcc_y_lane{lane}.csv", "w") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(post_vcc_y_lanes[lane])
                print(f"Wrote histogram data to {post_vcc_folder}post_vcc_y.csv")

    # Post-16k
    if post_16k_collected:
        post_16k_channelizer_folder.mkdir(parents=True, exist_ok=True)
        for lane in lanes:
            with open(
                str(post_16k_channelizer_folder) + f"/post_16k_x_lane{lane}.csv", "w"
            ) as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(post_16k_x_lanes[lane])
            with open(
                str(post_16k_channelizer_folder) + f"/post_16k_y_lane{lane}.csv", "w"
            ) as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(post_16k_y_lanes[lane])

## 3 Statecount Collection

### 3.1 Collecting Statecount Data

The wideband statecount can also be collected. Currently this is collected as a single CSV for each board, with each state count/power spectrum as a row. To collect it, we simply read the values of the relevant attributes for each board.

In [ ]:
vcc_statecount_holder = [[] for x in range(4)]
for board in boards:
    # For each board, link to the wbstatecount under that talon
    device = f"talondx-00{board}/wbstatecount/state_count"
    talon_proxy = DeviceProxy(device)
    # Capture the state count
    talon_proxy.command_inout("state_count_capture")
    # For each of the pieces of data we want to collect, grab the value from the tango device
    # Create a list with it, along with a header text
    state_count_vec_1 = talon_proxy.read_attribute("state_count_vector_1").value
    lst = list(state_count_vec_1)
    lst.insert(0, "State Count Vector 1")
    state_count_vec_1 = np.asarray(lst)

    state_count_vec_2 = talon_proxy.read_attribute("state_count_vector_2").value
    lst = list(state_count_vec_2)
    lst.insert(0, "State Count Vector 2")
    state_count_vec_2 = np.asarray(lst)

    power_spectrum_1 = talon_proxy.read_attribute("psd_vector_1").value
    lst = list(power_spectrum_1)
    lst.insert(0, "Power Spectrum 1")
    power_spectrum_1 = np.asarray(lst)

    power_spectrum_2 = talon_proxy.read_attribute("psd_vector_2").value
    lst = list(power_spectrum_2)
    lst.insert(0, "Power Spectrum 2")
    power_spectrum_2 = np.asarray(lst)

    # Store all these in an array
    vcc_statecount_holder[board] = [
        state_count_vec_1,
        state_count_vec_2,
        power_spectrum_1,
        power_spectrum_2,
    ]

Run to check the data neatly formatted in-notebook

In [ ]:
print(tabulate(vcc_statecount_holder[1], tablefmt="simple"))

### 3.2 Storing Statecount Data

Store the statecount data in files for each board under the relevant folder:

In [ ]:
# For each board, write a file with rows of the data collected for that board
for board in boards:
    statecount_folder = path + f"/talondx-00{board}/statecounts/"
    folder = pathlib.Path(statecount_folder)
    folder.mkdir(parents=True, exist_ok=True)
    with open(statecount_folder + "talon-" + str(board) + "_statecounts.csv", "w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(vcc_statecount_holder[board])

## 4 Compress Data

If desired, compress the folder to a zip file for easier downloading/sharing.

In [ ]:
shutil.make_archive(namespace, "zip", path)